# Importing libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Loading dataset

In [2]:
filepath1 = '../Data/df_final_demo.txt' # Client Profiles
filepath2 = '../Data/df_final_web_data_pt_1.txt' # Digital Footprints, part 1
filepath3 = '../Data/df_final_web_data_pt_2.txt' # Digital Footprints, part 2
filepath4 = '../Data/df_final_experiment_clients.txt' # Experiment Roster 

# Loading datasets
df_clients = pd.read_csv(filepath1)
df2 = pd.read_csv(filepath2)
df3 = pd.read_csv(filepath3)
df_roster = pd.read_csv(filepath4)

# Merging part 1 and 2 of Digital Footprints
df_trace = pd.concat([df2, df3], axis=0)

# Data cleaning

#### Dropping null values

In [5]:
# Making a dataframe with the rows that only have client_id, and will be deleted, in case I need it in the future
df_clients_with_na =  df_clients[df_clients.isna().sum(axis=1) > 7]

In [12]:
# Making a new dataframe with deleted rows that have as values only the client_id
df_clients = df_clients.dropna(thresh = 8)

#### Filling null values

In [7]:
# Replacing the null values at df_roster with "Undefined"
df_roster["Variation"] = df_roster["Variation"].apply(str).replace('nan','Undefined')

#### Formatting

In [13]:
# Formatting df_roster columns as lowercase, to match all the other colomn names
df_roster.columns = df_roster.columns.map(str.lower)

In [16]:
df_clients.dtypes

client_id            object
clnt_tenure_yr        int64
clnt_tenure_mnth      int64
clnt_age            float64
gendr                object
num_accts             int64
bal                 float64
calls_6_mnth          int64
logons_6_mnth         int64
dtype: object

In [15]:
# "client_id" to be casted as string, no intention to to perform mathematical operations on it
df_clients.client_id = df_clients.client_id.apply(str)

# casting the columns that I have tested are not real float number to integers
df_clients[["clnt_tenure_yr","clnt_tenure_mnth","num_accts",
            "calls_6_mnth","logons_6_mnth"]] = df_clients[["clnt_tenure_yr","clnt_tenure_mnth","num_accts",
                                                           "calls_6_mnth","logons_6_mnth"]].map(int)

In [18]:
# "client_id" to be casted as string, no intention to to perform mathematical operations on it
df_roster.client_id = df_roster.client_id.apply(str)

In [24]:
# "client_id" to be casted as string, no intention to to perform mathematical operations on it
df_trace.client_id = df_trace.client_id.apply(str)

# casting date_time as datetime pandas object
df_trace.date_time = pd.to_datetime(df_trace.date_time,format='%Y-%m-%d %H:%M:%S')